In [1]:
def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X

# 1a)

In [ ]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt 
import pickle

X = loadmat("BreastCancer.mat")['X']
y = loadmat("BreastCancer.mat")['y']

X_100 = X[:100,:]
y_100 = y[:100,:]

lam = np.logspace(-8, np.log10(20), 100)

w_star = ista_solve_hot(X, y, lam)

print((X_100@w_star-y_100).shape)
coord1vals = []
coord2vals = []
for c in range(100):
    temp1 = np.linalg.norm(w_star[:,c], ord=1)
    coord1vals.append(temp1)
    temp2 = (((np.sum((X_100@w_star-y_100)[:,c]))**2))**0.5
    coord2vals.append(temp2)

plt.plot(coord1vals, coord2vals, 'r')
plt.xlabel("Norm of w_star")
plt.ylabel("Norm of residual")

The lower l1 norm represents a lower lambda value. Small values of lambda increase the squared error. Thus we obtain such a graph where for high norm of w* we have a lower norm of residual.

# 1b)

In [ ]:
def sparcity(w_arr):
    nonzeroCount = 0
    for i in w_arr:
        if i>10**-6:
            nonzeroCount += 1
    return nonzeroCount

nonzero_values = []
err = []

for i in range(100):
    nonzero_values.append(sparcity(w_star[:,i]))
    y_hat2 = np.sign(X_100@w_star[:,i:i+1])
    if np.all((y_hat2==0)):
        err.append(1)
        continue
    #err.append(error_rate(np.sign(X_100@w[:,i:i+1], y_train)))
    err.append(np.sum(np.abs(y_hat2-y_100))/2/195)
    
plt.plot(nonzero_values, err, 'r')
plt.xlabel("Number of Nonzero Coefficents")
plt.ylabel("Error Rate")

As the number of nonzero coefficients goes up, we get decreasing error rates. Intuitively, this makes perfect sense, as more and more data points are used in the calculation, and few a wasted as zero coefficients. Thus, the error rate tends to 0 at really high nonzero coefficients or low sparsity. 